# Lecture 5: SBOX CPA Attack

In [ ]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook

output_notebook()

In [ ]:
import numpy as np

SBOX = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def sbox(data):
    return SBOX[data]

sbox_vec = np.vectorize(sbox)

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [ ]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [ ]:
import securec
from securec import util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./5_sbox.c')

In [ ]:
import numpy as np

scope.default_setup()

def capture(pt, samples=500):
    scope.adc.samples = samples
    scope.arm()
    target.simpleserial_write(0x01, pt + random.randbytes(16))
    return np.array(util.capture())

In [ ]:
import random
import tqdm
import tqdm.notebook

trace_samples = 500
trace_nums = 500

traces = []
inputs = []
for _ in tqdm.notebook.tqdm(range(trace_nums)):
    input = bytes([random.randint(0, 255) for _ in range(16)])
    traces.append(capture(input, samples=trace_samples))
    inputs.append(input)
traces = np.array(traces)
attempts = np.array([list(a) for a in inputs])

In [ ]:
import math

def pearson_pointwise_multi(traces, intermediates):
    (n, t) = traces.shape
    (_, m) = intermediates.shape

    d_traces = traces - np.einsum('nt->t', traces, dtype='float64', optimize='optimal') / np.double(n)
    d_intermediates = intermediates - np.einsum('nm->m', intermediates, dtype='float64', optimize='optimal') / np.double(n)
    
    tmp1 = np.einsum('nm,nm->m', d_intermediates, d_intermediates, optimize='optimal')
    tmp2 = np.einsum('nt,nt->t', d_traces, d_traces, optimize='optimal')
    tmp = np.einsum('m,t->mt', tmp1, tmp2, optimize='optimal')
    denominator = np.sqrt(tmp)
    numerator = np.einsum('nm,nt->mt', d_intermediates, d_traces, optimize='optimal')

    return np.nan_to_num(numerator / denominator)

def plot_correlation_vs_traces(
    traces,
    attempts,
    key_index=0,
    plotpoints=100,
):
    # Compute data
    plotpoints = min(plotpoints, len(traces))
    data = np.zeros((256, plotpoints))
    intermediates = np.array([hw_vec(sbox_vec(attempts[:, key_index] ^ guess)) for guess in range(256)]).T
    for i in range(0, plotpoints):
        j = math.ceil(i / plotpoints * len(traces))
        data[:, i] = np.max(np.abs(pearson_pointwise_multi(traces[:j, :], intermediates[:j, :])), axis=1)
    
    source = {
        'xs': len(data) * [list(range(0, len(traces), math.ceil(len(traces) / plotpoints)))],
        'ys': [corr for corr in data],
        'legend': list(range(256)),
        'color': math.ceil(len(data) / 20) * bokeh.palettes.Category20_20,
    }
    # Create figure
    p = figure(sizing_mode='stretch_width', height=300, tooltips=[('char', '@legend'), ('corrleation', '$y')])
    p.multi_line(xs='xs', ys='ys', color='color', source=source)
    return p

In [ ]:
show(plot_correlation_vs_traces(traces, attempts))

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>

Implement different countermeasures to make the attack harder and see visualize the success:

1. Randomize the order of the sbox lookup loop.
2. Add a time jitter around the lookup.

</div>

In [ ]:
util.exit()